# 实习一:数据库应用案例设计
---
## 组员：陈萧白，凤大骅，万承志

我们的案例是B站，实习内容包括确定业务需求，设计ER图，关系表设计与业务功能实现四部分。

## 一、业务需求
* 用户交互：用户可以关注其他用户，并且可以与其他用户互发私信
* 动态发布与交互：用户可以发布动态，也可以看到其他用户的动态，并可以点赞、评论
* 直播系统：用户可以发起直播，也可以观看其他用户的直播，发送弹幕或送出礼物

## 二、ER图设计

### 1.确定实体
4个强实体：
* **用户**（<u>用户UID</u>，昵称，等级）
* **私信**（<u>私信ID</u>，私信内容，私信时间）
* **动态**（<u>动态ID</u>，动态内容，发布时间）
* **直播间**（<u>房间号</u>，直播间名称，人气）
* **礼物**（<u>礼物ID</u>，礼物名称，礼物价值）

2个弱实体：
* **评论**（<u>评论ID，动态ID</u>，评论内容，发布时间）
* **直播间弹幕**（<u>弹幕ID，房间号</u>，弹幕内容，发送时间）

### 2.实体联系
一对多：
* 用户发送私信，用户接收私信
* 用户发布动态，用户发出评论，评论属于动态
* 用户创建直播间（实际上是一对一的），用户发送直播间弹幕，弹幕属于直播间

多对多：
* 用户关注用户，有关注时间属性
* 用户点赞动态，有点赞时间属性
* 用户进入直播间，有发生时间属性

三元联系：
* 用户给直播间赠送礼物，有时间、数量属性

### 3.ER图
ER图如下（其中弱实体的分辨符使用虚椭圆框与下划线表示）
![ER图](ER.png)

## 三、关系表设计